In [1]:
import os
import re
import nltk
import pandas as pd
import numpy as np

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

regex resources: https://regexr.com/ and 
https://docs.python.org/3/library/re.html#

In [3]:
cd

/Users/andrew.vangilder


In [4]:
os.chdir('Desktop/Notebooks')

In [6]:
scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials/sheets-api-test-12bfaf791b73.json', scope)
gc = gspread.authorize(credentials)

In [7]:
spreadsheet_key = '1IfqJFR8yE8lcF-DJQ980UmscoxaZ9Rv1D8AdF4mwODU'

In [8]:
book = gc.open_by_key(spreadsheet_key)

In [9]:
worksheet = book.sheet1
crosswalk = worksheet.get_all_values()

In [10]:
spreadsheet_key = '1-SX2_WtGv-tORP-1PCXy29jacYxF7Sf572pHIiYcQeg'

In [11]:
book = gc.open_by_key(spreadsheet_key)

In [25]:
worksheet = book.sheet1
icd03 = worksheet.get_all_values()

In [ ]:
dataframe

In [27]:
def gsheet2df(gsheet):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!
    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.
    """
    header = gsheet[0]   # Assumes first line is header!
    values = gsheet[1:]  # Everything else is data.
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df


In [42]:
icd03_df = gsheet2df(icd03)
icd03_df = icd03_df.rename(index=str,columns={'Histology Behavior':'ICDO3'})

In [49]:
icd03_df.groupby('ICDO3').groups

{u'': Index([u'9368', u'9383', u'9405', u'9420'], dtype='object'),
 u'8000/0': Index([u'18606', u'18638', u'18842', u'18877', u'19157', u'19193', u'19385',
        u'19416', u'19641', u'19677', u'20433', u'20466', u'20565', u'20598',
        u'20693', u'20730'],
       dtype='object'),
 u'8000/1': Index([u'18607', u'18639', u'18843', u'18878', u'19158', u'19194', u'19386',
        u'19417', u'19642', u'19678', u'20434', u'20467', u'20566', u'20599',
        u'20694', u'20731'],
       dtype='object'),
 u'8000/3': Index([u'0', u'76', u'222', u'366', u'526', u'711', u'886', u'1064', u'1244',
        u'1376',
        ...
        u'20695', u'20732', u'20773', u'20806', u'20881', u'20901', u'20921',
        u'20959', u'21067', u'21106'],
       dtype='object', length=137),
 u'8001/0': Index([u'18609', u'18641', u'18845', u'18880', u'19160', u'19196', u'19388',
        u'19419', u'19644', u'19680', u'20436', u'20469', u'20568', u'20601',
        u'20696', u'20733'],
       dtype='object'),
 

In [43]:
crosswalk_df = gsheet2df(crosswalk)

In [51]:
crosswalk_df.groupby('ICDO3').groups

{u'': Int64Index([   164,   1328,   1997,   2274,   2874,   3065,   3221,   3869,
               3973,   3991,
             ...
             180454, 181053, 181563, 181871, 182656, 184917, 185148, 186783,
             186887, 187211],
            dtype='int64', length=321),
 u'8471/3': Int64Index([ 15007,  19202,  31782,  52239,  65217,  73157,  73799,  92825,
              94402,  94487,  94892,  94999, 100594, 104529, 109546, 109808,
             113175, 122857, 124838, 126011, 126625, 126774, 136712, 142256,
             142451, 142838, 143092, 143832, 144595, 153489, 175222, 176872,
             177160, 179940, 182978],
            dtype='int64'),
 u'8471/0': Int64Index([1566], dtype='int64'),
 u'8813/0': Int64Index([62921], dtype='int64'),
 u'9351/0': Int64Index([32947, 105468, 141795], dtype='int64'),
 u'9581/3': Int64Index([  1210,   2524,   7728,  14019,  14667,  15564,  23525,  23851,
              26634,  30427,
             ...
             155828, 165059, 170619, 170628, 17

In [54]:
crosswalk_df[crosswalk_df['ICDO3']=='9065/3'].head()

,ICDO3,ICD10,ICDO3 Display,SNOMEDCT,SNOMEDCT Display
423,9065/3,C74.9,"Germ cell tumor, nonseminomatous of adrenal gland",23451007,Adrenal structure
3574,9065/3,C38.2,"Germ cell tumor, nonseminomatous of posterior ...",93969002,Primary malignant neoplasm of posterior medias...
4735,9065/3,C80.9,"Germ cell tumor, nonseminomatous of unknown pr...",145831000119103,Primary malignant germ cell neoplasm
7452,9065/3,C38.3,"Germ cell tumor, nonseminomatous of mediastinum",72410000,Mediastinal structure
8918,9065/3,C75.2,"Germ cell tumor, nonseminomatous of craniophar...",86394004,Structure of craniopharyngeal duct


In [57]:
icd03_df[icd03_df['ICDO3']=='9065/3']

,Site Recode,Site Description,Histology,Histology Description,ICDO3,Histology Behavior/Description
4505,"C260,C268-C269",UNSPECIFIED DIGEST. ORGANS,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
4538,"C260,C268-C269",UNSPECIFIED DIGEST. ORGANS,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
7684,"C381-C383,C388",MEDIASTINUM,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
7717,"C381-C383,C388",MEDIASTINUM,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
10368,"C480-C482,C488",RETROPERITONEUM & PERITONEUM,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
10402,"C480-C482,C488",RETROPERITONEUM & PERITONEUM,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
13617,C569,OVARY,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
13650,C569,OVARY,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
15127,"C620-C621,C629",TESTIS,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"
15164,"C620-C621,C629",TESTIS,906,GERM CELL TUMORS,9065/3,"Germ cell tumor, nonseminomatous"


In [44]:
head1 = icd03_df.head()
head2 = crosswalk_df.head()

In [45]:
head1

,Site Recode,Site Description,Histology,Histology Description,ICDO3,Histology Behavior/Description
0,"C000-C006,C008-C009",LIP,800,NEOPLASM,8000/3,"Neoplasm, malignant"
1,"C000-C006,C008-C009",LIP,800,NEOPLASM,8001/3,"Tumor cells, malignant"
2,"C000-C006,C008-C009",LIP,800,NEOPLASM,8002/3,"Malignant tumor, small cell type"
3,"C000-C006,C008-C009",LIP,800,NEOPLASM,8003/3,"Malignant tumor, giant cell type"
4,"C000-C006,C008-C009",LIP,800,NEOPLASM,8004/3,"Malignant tumor, spindle cell type"


In [46]:
head2

,ICDO3,ICD10,ICDO3 Display,SNOMEDCT,SNOMEDCT Display
0,8850/3,C18.3,Liposarcoma of hepatic flexure of colon,48338005,Structure of right colic flexure
1,9727/3,C25.8,Precursor cell lymphoblastic lymphoma of overl...,372003004,Primary malignant neoplasm of pancreas
2,9738/3,C25.9,Large B-cell lymphoma arising in HHV8-associat...,372003004,Primary malignant neoplasm of pancreas
3,9560/0,C30.1,Neurilemoma of middle ear,985004,Neurilemoma
4,9560/0,C47.0,Neurilemoma of peripheral nerves and autonomic...,985004,Neurilemoma


In [48]:
icd03_df.merge(crosswalk_df,how='right',on='ICDO3')

,Site Recode,Site Description,Histology,Histology Description,ICDO3,Histology Behavior/Description,ICD10,ICDO3 Display,SNOMEDCT,SNOMEDCT Display
0,"C000-C006,C008-C009",LIP,800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
1,C019,BASE OF TONGUE,800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
2,"C020-C024,C028-C029",TONGUE (EXCL BASE OF TONGUE),800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
3,"C030-C031,C039-C041,C048-C052,C058-C062,C068-C069","GUM, FLOOR OF MOUTH, & OTHER MOUTH",800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
4,"C079-C081,C088-C089",SALIVARYGLAND,800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
5,"C090-C091,C098-C104,C108-C109",OROPHARNYX,800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
6,"C110,C112-C113,C118-C119",NASOPHARYNX(EXCL POSTERIOR WALL),800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
7,C111,POSTERIOR WALL OF NASOPHARYNX,800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
8,"C129-C132,C138-C139",HYPOPHARYNX,800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas
9,"C140,C142,C148",PHARYNX,800,NEOPLASM,8000/3,"Neoplasm, malignant",C25.9,"Neoplasm, malignant of pancreas",372003004,Primary malignant neoplasm of pancreas


In [311]:
#cleanest_df.to_csv("data/andrew_test.csv")